In [ ]:
import pandas as pd
import openai
import pinecone


In [ ]:
!pip install openai pinecone LangChain

In [ ]:
!pip install --upgrade openai

In [ ]:
# OpenAI API 키 설정
from pinecone import Pinecone
from openai import OpenAI


openai.api_key = 'sk-proj-p2ti3uV1OWTRF3d5IljCT3BlbkFJGUHpeTHBINuJDMP5x5l3'



pc = Pinecone(api_key="6a58b4b8-48f2-49aa-952f-5e4658e23f69")
index = pc.Index("prod")

In [ ]:
def embedding(data):
    response = openai.embeddings.create(
        input= data,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding


In [ ]:
#생성한 목 데이터를 업로드합니다.
index.upsert(
    vectors=vectors,
    namsespace="loadmap_embed_with_summary")

In [ ]:
query = "AWS 서비스명: AWS IoT Core, Amazon Kinesis, Amazon QuickSight, AWS LambdaIT 기술 키워드: 데이터 수집, 대시보드 개발, 데이터 시각화, 이벤트 기반 아키텍처"


print(embedding(query))


In [ ]:
def get_aws_and_it_keywords(user_question):
    prompt = f"""
    The user asked the following question:
    "{user_question}"

    Provide four AWS service names and four IT technology keywords related to the question.
    Each keyword should be separated by a comma and a space, and the whole output should be in one line.
    Do not include any labels such as "AWS services" or "IT keywords.
    Exclude "IAM" from the AWS services."
    """
    response =  openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specializing in AWS and IT technology."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0
    )
    return response.choices[0].message.content



In [ ]:
def get_aws_and_it_keywords_from_desc(desc):
    prompt = f"""
    The user asked the following question:
    "{desc}"

    Provide four AWS service names and four IT technology keywords related to the content.
    Each keyword should be separated by a comma and a space, and the whole output should be in one line.
    Do not include any labels such as "AWS services" or "IT keywords.
    Exclude "IAM" from the AWS services."
    """
    response =  openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specializing in AWS and IT technology."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0
    )
    return response.choices[0].message.content



In [ ]:
df = pd.read_csv('/content/roadmap_38_no_keywords-UTF.csv')


selected_cols = df[['id','title','desc','thumbnail', 'service','job', 'summary', 'keywords' ]]



## 그냥 그대로 summary만 임베딩하여 넣었을 경우
vectors = []
for i, row in selected_cols.iterrows():

    vector = {}
    metadata = {}

    vector["metadata"] = metadata
    vector["values"] = ""
    print(i)
    try:
    # sum = get_summary(row["desc"])
        keywords = get_aws_and_it_keywords_from_desc(row["desc"])
        # selected_cols.at[i, 'summary'] = sum
        keywords = keywords.replace("Amazon","").replace("AWS", "").replace("  ", " ")
        selected_cols.at[i, 'keywords'] = keywords
    except Exception as e:


        selected_cols.to_csv('./updated_roadmap.csv', index=False)


    #vectors.append(vector)

selected_cols.to_csv('./updated_roadmap.csv', index=False)








In [ ]:
query = get_aws_and_it_keywords("채팅서버 만들어보고 싶어")

print(query)

print(embedding(query))




In [ ]:
pc.delete_index("roadmap-test")

In [ ]:
df = pd.read_csv('/content/updated_roadmap_38_final.csv')


selected_cols = df[['id','title','desc','thumbnail', 'service','job', 'summary', 'keywords' ]]
vectors = []
for i, row in selected_cols.iterrows():
    vector = {}
    metadata = {}
    vector["id"] = str(row['id'])+"_roadmap_with_summary_keywords"
    embedding_data = embedding(row["summary"]+" "+row["keywords"])
    metadata["id"] = str(row['id'])
    vector["metadata"] = metadata
    vector["values"] = embedding_data
    vectors.append(vector)

print(vectors[0])

In [ ]:
#생성한 목 데이터를 업로드합니다.
index.upsert(
    vectors=vectors,
    namespace="roadmapSummaryKeyword"
)

In [ ]:
print(vectors)